In [29]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jul  1 13:16:32 2021

@author: gnate
"""

import numpy as np

# Define some noise functions
def zero_bit_noise(N):
    noise_list = []
    noise_vec = np.zeros((1,N))
    noise_list.append(noise_vec)
    return noise_list

def one_bit_noise(N):
    noise_list = []
    for i in range(0,N):
        noise_vec = np.zeros((1,N))
        noise_vec[0,i] = 1
        noise_list.append(noise_vec)
    return noise_list

def two_bit_noise(N):
    noise_list = []
    for i in range(0,N):
        noise_vec = np.zeros((1,N))
        noise_vec[0,i] = 1     
        for j in range(i+1,N):
            temp_vec = np.copy(noise_vec)
            temp_vec[0,j] = 1
            noise_list.append(temp_vec)
    return noise_list
 
def xor_array(x,y,dim):
    a = x.reshape((1,dim))
    b = y.reshape((1,dim))
    output_vec = np.zeros((1,a.shape[1]), dtype = 'uint')
    for i in range(0, a.shape[1]):
        output_vec[0, i] = a[0, i] ^ b[0, i]
    return output_vec        

# Define translation functions    
def no_translation(x):
    return x

def up_translation(x):
    dim = np.sqrt(x.shape[1])
    x_image = x.reshape((int(dim), int(dim)))
    output_image = np.zeros((int(dim), int(dim)), dtype = 'uint')    
    output_image = np.roll(x_image, -1, axis=0) #up
    return output_image.reshape(1, x.shape[1])

def down_translation(x):
    dim = np.sqrt(x.shape[1])
    x_image = x.reshape((int(dim), int(dim)))
    output_image = np.zeros((int(dim), int(dim)), dtype = 'uint')    
    output_image = np.roll(x_image, 1, axis=0)  #down
    return output_image.reshape(1, x.shape[1])

def left_translation(x):
    dim = np.sqrt(x.shape[1])
    x_image = x.reshape((int(dim), int(dim)))
    output_image = np.zeros((int(dim), int(dim)), dtype = 'uint') 
    output_image = np.roll(x_image, -1, axis=1)  #left
    return output_image.reshape(1, x.shape[1])

def right_translation(x):
    dim = np.sqrt(x.shape[1])
    x_image = x.reshape((int(dim), int(dim)))
    output_image = np.zeros((int(dim), int(dim)), dtype = 'uint') 
    output_image = np.roll(x_image, 1, axis=1)  #left
    return output_image.reshape(1, x.shape[1])

def top_left_diag(x):
    dim = np.sqrt(x.shape[1])
    x_image = x.reshape((int(dim), int(dim)))
    output_image = np.zeros((int(dim), int(dim)), dtype = 'uint') 
    output_image = up_translation(x_image.reshape((1,x.shape[1]))).reshape((int(dim), int(dim)))
    output_image = left_translation(output_image.reshape((1,x.shape[1]))).reshape((int(dim), int(dim)))
    return output_image.reshape(1, x.shape[1])

def top_right_diag(x):
    dim = np.sqrt(x.shape[1])
    x_image = x.reshape((int(dim), int(dim)))
    output_image = np.zeros((int(dim), int(dim)), dtype = 'uint') 
    output_image = up_translation(x_image.reshape((1,x.shape[1]))).reshape((int(dim), int(dim)))
    output_image = right_translation(output_image.reshape((1,x.shape[1]))).reshape((int(dim), int(dim)))
    return output_image.reshape(1, x.shape[1])

def bottom_left_diag(x):
    dim = np.sqrt(x.shape[1])
    x_image = x.reshape((int(dim), int(dim)))
    output_image = np.zeros((int(dim), int(dim)), dtype = 'uint') 
    output_image = down_translation(x_image.reshape((1,x.shape[1]))).reshape((int(dim), int(dim)))
    output_image = left_translation(output_image.reshape((1,x.shape[1]))).reshape((int(dim), int(dim)))
    return output_image.reshape(1, x.shape[1])

def bottom_right_diag(x):
    dim = np.sqrt(x.shape[1])
    x_image = x.reshape((int(dim), int(dim)))
    output_image = np.zeros((int(dim), int(dim)), dtype = 'uint') 
    output_image = down_translation(x_image.reshape((1,x.shape[1]))).reshape((int(dim), int(dim)))
    output_image = right_translation(output_image.reshape((1,x.shape[1]))).reshape((int(dim), int(dim)))
    return output_image.reshape(1, x.shape[1])

def translation_pkg(x):
    dim = np.sqrt(x.shape[1])
    all_translation = np.zeros((9, 1, x.shape[1]))
    all_translation[0] = no_translation(x)
    all_translation[1] = up_translation(x)
    all_translation[2] = down_translation(x)
    all_translation[3] = left_translation(x)
    all_translation[4] = right_translation(x)
    all_translation[5] = top_left_diag(x)
    all_translation[6] = top_right_diag(x)
    all_translation[7] = bottom_left_diag(x)
    all_translation[8] = bottom_right_diag(x)
    return all_translation

def hamming_dist(x,y,dim):
    dist = 0
    output_vec = xor_array(x,y,dim)
    dist = sum(output_vec.T)
    return dist

## Main program ##
#Generate 3x3 noiseless data
N = 25 #25
orig_inputs = ['1111100010001000100011111', '1000101010010100010000100', '1000111001101011001110001'] #z v n in 5x5
orig_inputs_matrix = np.zeros((len(orig_inputs), 1, N), dtype = 'uint')
for i in range(0,len(orig_inputs)):
    input_tuple = tuple(orig_inputs[i])
    for j in range(0, N):
        if input_tuple[j] == '1': orig_inputs_matrix[i,0,j] = 1
#print('Input Matrix -', orig_inputs_matrix)

# Generate 5x5 noisy data
noise_vectors = []
noise_vectors.append(zero_bit_noise(N))
noise_vectors.append(one_bit_noise(N))

count_vec = np.zeros(2, dtype = 'uint')
for i in range(0,2):
    count_vec[i] = np.math.factorial(N)/(np.math.factorial(i)*np.math.factorial(N-i))

# Generate 3x3 noisy data
noisy_data_matrix = np.zeros((int(len(orig_inputs)*sum(count_vec)), 1, N), dtype = 'uint')
row_count = 0
for i in range(0,3):
    for j in range(0,2):
        bit_noise = np.array(noise_vectors[j], dtype = 'uint')
        for k in range(0, int(count_vec[j])):
            noisy_data_matrix[row_count] = xor_array(orig_inputs_matrix[i], bit_noise[k], N)
            row_count += 1
               
# # Implant 3x3 noisy data into 5x5 
# M = 25
# sqrt_M = np.sqrt(M)
# aug_data_matrix = np.zeros((len(orig_inputs)*sum(count_vec), 1, M), dtype = 'uint')
# 
# for row in range(0, len(orig_inputs)):
#     orig_temp_matrix = orig_inputs_matrix[row].reshape((3,3))
#     aug_temp_matrix =  np.zeros((int(sqrt_M), int(sqrt_M)), dtype = 'uint')
#     for i in range(0,3):
#         for j in range(0,3):
#             aug_temp_matrix[i+1, j+1] = orig_temp_matrix[i,j]
#     aug_data_matrix[row] = aug_temp_matrix.reshape((1,M))
# 
# # Generate translations in 5x5  
# translated_data_matrix = np.zeros((9*len(orig_inputs)*sum(count_vec), 1, M), dtype = 'uint')
# for row in range(0, len(orig_inputs)*sum(count_vec)):
#     translated_data_matrix[9*row: 9*(row+1)] = translation_pkg(aug_data_matrix[row])

            
# Generate final matrix to be saved
final_data_matrix = np.zeros((int(9*len(orig_inputs)*sum(count_vec)), 1, N+1), dtype = 'uint')
final_data_matrix[0:9*int(len(orig_inputs)*sum(count_vec)), 0, 1:N+1] = noisy_data_matrix.reshape(9*int(len(orig_inputs)*sum(count_vec)), N)

# Add labels
for i in range(0,len(orig_inputs)):
    final_data_matrix[9*sum(count_vec)*i: 9*sum(count_vec)*(i+1), 0, 0] = i
    
# Generate distance matrix
distance_matrix = np.zeros((len(orig_inputs), len(orig_inputs)))
temp_distance = np.zeros(9*sum(count_vec))
for i in range(0, len(orig_inputs)):
    for k in range(i, len(orig_inputs)):
        for j in range(0, 9*sum(count_vec)):
            temp_distance[j] = hamming_dist(aug_data_matrix[i], noisy_data_matrix[j+(k*9*sum(count_vec))], N)
            
        if k == i: distance_matrix[i, k] = max(temp_distance)
        else: 
            distance_matrix[i, k] = min(temp_distance)
            distance_matrix[k, i] = distance_matrix[i, k]

print('Distance matrix - \n', distance_matrix)
 
# # Save data as csv file
np.savetxt("5x5image_translations.csv", final_data_matrix.reshape(9*len(orig_inputs)*sum(count_vec), M+1), delimiter=",")

ValueError: cannot reshape array of size 1950 into shape (702,25)

In [47]:
noisy_data_matrix.shape

(78, 1, 25)

In [49]:
final_data_matrix.shape

(702, 1, 26)